## Pre-Training [`GPT-Neo`](https://github.com/huggingface/transformers/blob/main/src/transformers/models/gpt_neo/modeling_gpt_neo.py) & [`RoBERTa`](https://github.com/huggingface/transformers/blob/main/src/transformers/models/roberta/modeling_roberta.py) on [`TinyStories`](https://huggingface.co/datasets/roneneldan/TinyStories)

This script shows how to pre-train both models. I put them in one notebook because the majority of the code is shared; but you may want to separate the logic per model. 

In [34]:
import wandb; wandb.login()
from transformers import (
    RobertaForMaskedLM, RobertaConfig, RobertaTokenizerFast,
    GPTNeoForCausalLM, GPTNeoConfig, GPT2TokenizerFast
)

#### Models 
We consider GPT-Neo and BERT as base transformer architectures. This consists of the following blocks linked via residual connections:

- Embeddings: Map one-hot (sparse) token vectors to a dense vector of length `hidden_size`. Add positional encoding. 
- $n$ Blocks: Contain self-attention and FFNs.
- Head: Map hidden state back to a useful output. 

This specifies some of the model-related hyperparameters. I chose them based on what achieved reasonable performance in the [TinyStories paper](https://arxiv.org/abs/2305.07759), while also being feasible to train on our limited compute budgets. 

In [52]:
config_gpt = dict(

    # EMBEDDING PARAMETERS
    vocab_size              = 10_000,   # number of tokens in the vocabulary 
    hidden_size             = 516,      # embedding size (vector length) of each token 
    max_position_embeddings = 512,      # maximum sequence length (context window)

    # BLOCKS (ATTN & FFN)
    num_layers=2,                               # number of transformer blocks
    attention_types=[[["global", "local"], 1]], # (GPT-Neo-specific) global and local attention 
    num_heads=4,                                # attention heads
    window_size=256,                            # (GPT-Neo-specific) for local attention 
    intermediate_size=1024,                     # size of 'up-projection' layer in FFN

    pad_token_id = 1,           # we need to specify this for GPT Neo for tokenizer interop
)

config_rob = dict(
    
    # EMBEDDING PARAMETERS
    vocab_size              = 10_000,   # number of tokens in the vocabulary 
    hidden_size             = 516,      # embedding size (vector length) of each token 
    max_position_embeddings = 512,      # maximum sequence length (context window)

    # BLOCKS (of course naming is different in roberta :) )
    num_hidden_layers = config_gpt['num_layers'],
    num_attention_heads = config_gpt['num_heads'],
    intermediate_size=1024,                     # size of 'up-projection' layer in FFN
)

config_gpt = GPTNeoConfig(**config_gpt)
config_rob = RobertaConfig(**config_rob)

In [58]:
# TODO: you should set all pytorch & huggingface seeds here as model initialisation depends on it

gpt = GPTNeoForCausalLM(config=config_gpt)
rob = RobertaForMaskedLM(config=config_rob)

print(f'''
    This GPT has {gpt.num_parameters():,} parameters,
     and ROB has {rob.num_parameters():,} parameters.
    ''')

gpt, rob # uncomment to see model architecture


    This GPT has 9,677,048 parameters,
     and ROB has 9,958,980 parameters.
    


#### Pre-Processing
All experiments use the same tokenizer, so in theory, we only need to preprocess our data once. This means you can subsequently skip this CPU-intensive task on the DelftBlue GPU nodes. 
(except if you are experimenting with different context lengths, in that case you will need to re-tokenise to recover potentially truncated inputs; see below).

When running experiments on delftblue, it may be fastest to load (parts of) the dataset into memory, as the cluster has pretty slow IO. See `datasets`, specifically [`load_dataset`'s `keep_in_memory` attribute](https://huggingface.co/docs/datasets/en/package_reference/loading_methods#datasets.load_dataset). 

In [72]:
import tqdm 
from datasets import load_dataset

dataset = load_dataset('roneneldan/tinystories')

n_words = 0 
for story in tqdm.tqdm(dataset['train']['text']):    # tqdm is used for progress bars around iterables
    n_words += len(story.split())

print(f'''
    Train dataset has {len(dataset['train']['text']):,} samples, 
    totalling {n_words:,} words (avg. {n_words/len(dataset['train']['text']):,.1f} per story).
    ''')

/Users/Aral/anaconda3/envs/lms/lib/python3.11/site-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")
100%|██████████| 2119719/2119719 [00:11<00:00, 179830.98it/s]



    Train dataset has 2,119,719 samples, 
    totalling 371,713,604 words (avg. 175.4 per story).
    


#### Tokenisation
Two things happen during tokenisation, of which the latter is optional:

1. Map (sub-)words to integers $ \in [0, 10000]$.
2. Truncate/pad each input as necessary to fit within `hidden_size`. 

All it does is map frequent character strings (i.e. tokens) to integers, which it can then use to look up the corresponding embedding vector for that integer index.

The intuition is that by assigning each token its own embedding (vector of `1 x hidden_size`) in the transformer, will help it learn semantical understanding of these tokens. E.g. say that you have two tokens 'about' and 'around', the transformer should learn similar embedding vectors for each. 

In [77]:
from transformers import PreTrainedTokenizer, PretrainedConfig

def get_tokenizer_for_config(Tok: PreTrainedTokenizer, config: PretrainedConfig):

    tokenizer = Tok.from_pretrained(
        '10k-tok',                                         # our custom tokenizer
        model_max_length=config.max_position_embeddings    # sequence length (context window)
    )

    # we're using our special tokenizer with only 10'000 tokens instead of 50'256
    assert tokenizer.vocab_size == config.vocab_size

    print(f'padding token is {tokenizer.pad_token}')
    print(f'padding token in config: {config.pad_token_id}, in tokeniser: {tokenizer.pad_token_id}')
    
    return tokenizer 

tok_gpt = get_tokenizer_for_config(GPT2TokenizerFast, config_gpt)
tok_rob = get_tokenizer_for_config(RobertaTokenizerFast, config_rob)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RobertaTokenizer'. 
The class this function is called from is 'GPT2TokenizerFast'.


padding token is <pad>
padding token in config: 1, in tokeniser: 1
padding token is <pad>
padding token in config: 1, in tokeniser: 1


In [74]:
# tokenized_gpt = dataset.map(
#     lambda x: tok_gpt(x['text'], truncation=True, padding='max_length'), 
#     batched=True, num_proc=5, batch_size=1_000)

# tokenized_rob = dataset.map(
#     lambda x: tok_rob(x['text'], truncation=True, padding='max_length'), 
#     batched=True, num_proc=5, batch_size=1_000)

# # sanity check that the tokenisers are indeed identical 
# for split in ['train', 'validation']:
#     for sample_gpt, sample_rob in tqdm.tqdm(zip(tokenized_gpt[split], tokenized_rob[split])):
#         assert sample_gpt == sample_rob

# tokenized_gpt.save_to_disk('./tokenized_dataset', num_proc=5)

In [86]:
from datasets import load_from_disk 
tokenized_dataset = load_from_disk(f'./tokenized_dataset')

train_dataset = tokenized_dataset['train']
eval_dataset  = tokenized_dataset['validation']

assert len(tokenized_dataset['train'][0]['input_ids']) == config_gpt.max_position_embeddings
tokenized_dataset['train'][0]['input_ids'][-10:] 
# should be pad tokens (1), given that most short stories are <512 tokens

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

#### Training

Huggingface provides some powerful (and often confusingly long) APIs for model training. The `TrainingArguments` specifies our hyperparameters, which are used by the `Trainer` taking in the remaining objects (like `model`, `tokenizer`, and `train_dataset`). Specifically:

- `learning_rate` and `num_train_epochs` determine how much the model learns. A higher rate is faster, but more unstable. More epochs (entire passes over the dataset) yields incrementally better results, at the cost of more training time. 
- Batch sizes determine how many samples the model sees in *parallel*. Given `gradient_accumulation_steps=1` and a `batch_size=8`, the model will backpropagate the average loss of 8 samples; if `batch_size=1`, it will average the loss of `gradient_accumulation_steps` samples. It is important to make sure the backpropagated loss is averaged over the same number of samples, when comparing models. 

- `data_collator` batches (and optionally, pads) the input for the model. We have already padded in our `tokenized_dataset`, and leaving this argument empty will automatically batch the inputs. So why do we need it? 

    Glad you asked. This has to do with how the loss is computed in causal language modelling. In our case, we try to predict $p(y | x)$, where $x$ is an input sequence of tokens, and $y$ is the next token following that sequence. Our model, unaware of the target token $y$, outputs $\hat y$. 
    
    For `Trainer` to compute the (cross-entropy) loss, we need to provide it with both $y$ and $\hat y$. The `DataCollatorForLanguageModeling` knows this, and provides the next token $y$ as a separate part of the input, to the `Trainer`.

    The loss is the backbone of backpropagation, which we need to actually improve our model. If this is confusing, please re-watch Karpathy's GPT tutorial. 

If you prefer to write the training loop yourself, check out HF' `run_clm_no_trainer.py` scripts. (`run_mlm_no_trainer.py` for RoBERTa-style masked-language modelling, as opposed to causal language modelling). This can be useful to give you better control over which devices are used for training. 

In [125]:
from dataclasses import dataclass  # i like using dataclasses for modularity
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

def get_hyperparameters(model, dataset):
    ''' common hyperparameters to give to the trainer '''

    # TRAINING HYPERPARAMETERS 
    batch_size = 16                  # TinyStories uses 80, but I am training locally on my poor M1 Air
    num_train_epochs = 1             # TinyStories doesn't mention
    gradient_accumulation_steps = 16 # TinyStories uses 16

    lr = 5e-4                        # TinyStories uses 5e-4, higher values better for small models

    # future you will thank you for descriptive model names
    config      = model.config 
    model_name  = '-'.join([
        'GPT' if isinstance(model, GPTNeoForCausalLM) else 'BERT',
        f'{model.num_parameters()//1e6:.1f}M',
        f'{config.num_layers if isinstance(model, GPTNeoForCausalLM) else config.num_hidden_layers}L', 
        f'{config.num_heads if isinstance(model, GPTNeoForCausalLM) else config.num_attention_heads}H', 
        f'{config.hidden_size}C',
        f'{config.intermediate_size}I'
    ])

    _train_steps = len(dataset) // (batch_size * gradient_accumulation_steps)
    eval_steps = _train_steps // 10 # evaluate every 10% of training steps

    return dict(
        model_name = model_name,
        batch_size = batch_size, 
        num_train_epochs = num_train_epochs,
        gradient_accumulation_steps = gradient_accumulation_steps,
        lr = lr,
        eval_steps = eval_steps
    )

params_gpt = get_hyperparameters(gpt, train_dataset)
params_rob = get_hyperparameters(rob, train_dataset)

In [126]:
def get_trainer(
        model, tokenizer, train_dataset, eval_dataset,
        model_name, batch_size, num_train_epochs, gradient_accumulation_steps, lr, eval_steps):
    ''' more general training arguments you likely want to keep fixed'''

    training_args = TrainingArguments(

        seed       = 42,
        use_cpu    = False, # use GPU if available (not necessarily faster on laptops, but Apple's MPS have good support)

        output_dir = f'./results/models/{model_name}',

        # NOTE: training params
        learning_rate    = lr,
        num_train_epochs = num_train_epochs,
        # Use a smaller batch size to fit into GPU RAM. 
        per_device_train_batch_size = batch_size,
        per_device_eval_batch_size  = batch_size,
        # You should aim to have the same amount of samples per acc step, in all of your experiments!
        # so, if you increase batch_size, decrease gradient_accumulation_steps by the same factor.
        gradient_accumulation_steps = gradient_accumulation_steps,

        # NOTE: Evaluation params
        # wandb is great for tracking experiments, it will even (try to) save your code nowadays
        evaluation_strategy = 'steps',
        eval_steps = eval_steps,
        save_steps = eval_steps,

        logging_first_step=True,
        logging_steps=eval_steps,
        report_to  = 'wandb',
    )

    trainer = Trainer(
        model = model, 
        args = training_args, 
        train_dataset = train_dataset, 
        eval_dataset = eval_dataset,
        data_collator = DataCollatorForLanguageModeling(
            tokenizer, mlm=isinstance(model, RobertaForMaskedLM)),
    )

    # print amount of training steps, and how often the model is evaluated
    print(f'''
    Retrieving Trainer for \033[1m{model_name}\033[0m
        training for {num_train_epochs} epochs, {len(train_dataset)} samples
        {batch_size} batch size, {gradient_accumulation_steps} accumulation steps
        gives {len(train_dataset)//(batch_size * gradient_accumulation_steps)} training steps.
        Evaluating every {eval_steps} steps, {len(eval_dataset)} samples 
        ''')

    return trainer

In [127]:
trainer_gpt = get_trainer(gpt, tok_gpt, train_dataset, eval_dataset, **params_gpt)
trainer_rob = get_trainer(rob, tok_rob, train_dataset, eval_dataset, **params_rob)


    Retrieving Trainer for GPT-9.0M-2L-4H-516C-1024I
        training for 1 epochs, 2119719 samples
        16 batch size, 16 accumulation steps
        gives 8280 training steps.
        Evaluating every 828 steps, 21990 samples 
        

    Retrieving Trainer for BERT-9.0M-2L-4H-516C-1024I
        training for 1 epochs, 2119719 samples
        16 batch size, 16 accumulation steps
        gives 8280 training steps.
        Evaluating every 828 steps, 21990 samples 
        


Finally, we can train. 

This configuration takes ≤24hr to pre-train on my M1 Macbook Air with 16GB RAM. Python takes ≤4GB VRAM at a `batch_size=16` and ≤11GB at `batch_size=64`, though they take the same amount of time to train - likely because this processor is not designed to move that much data in and out of RAM constantly. And tbh, the GPU be lacking. If you decide to go the local-training-route, consider [chai](https://github.com/lvillani/chai) to keep your (Apple) laptop awake – there's probably a windows/linux equivalent too. 

In [128]:
def do_train(trainer, name): 

    wandb.init(project='tinystories', name=name, config=trainer.args)
    trainer.train()
    trainer.save_model(f'./results/models/{name}')

In [129]:
trainer_gpt.model

GPTNeoForCausalLM(
  (transformer): GPTNeoModel(
    (wte): Embedding(10000, 516)
    (wpe): Embedding(512, 516)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-1): 2 x GPTNeoBlock(
        (ln_1): LayerNorm((516,), eps=1e-05, elementwise_affine=True)
        (attn): GPTNeoAttention(
          (attention): GPTNeoSelfAttention(
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
            (k_proj): Linear(in_features=516, out_features=516, bias=False)
            (v_proj): Linear(in_features=516, out_features=516, bias=False)
            (q_proj): Linear(in_features=516, out_features=516, bias=False)
            (out_proj): Linear(in_features=516, out_features=516, bias=True)
          )
        )
        (ln_2): LayerNorm((516,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTNeoMLP(
          (c_fc): Linear(in_features=516, out_features=1024, bias=True)
          (c_proj): Linear(in_featu

In [130]:
do_train(trainer_gpt, params_gpt['model_name'])

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


  0%|          | 0/8280 [00:00<?, ?it/s]

{'loss': 4.3356, 'grad_norm': 1.180248737335205, 'learning_rate': 0.0004999396135265701, 'epoch': 0.0}


#### Using your Pre-Trained Model 
Try out your own model on some prompts!

In [ ]:
import textwrap # for pretty printing
w = textwrap.TextWrapper(replace_whitespace=False, break_long_words=False, width=60, initial_indent='   ', subsequent_indent='  ')
def see(text): print('\n\033[3m' + '\n\n'.join(['\n'.join(w.wrap(line))
                 for line in text.splitlines() if line.strip() != '']) + '\033[0m\n')

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = 'GPT-9.0M-2L-4H-512C-1024I'

tokenizer = AutoTokenizer.from_pretrained('10k-gpt-neo')
tokenizer.pad_token_id = tokenizer.eos_token_id
model = AutoModelForCausalLM.from_pretrained(f'results/models/{model_name}')

In [ ]:
prompt = 'Once upon a time, there was a chicken with'
input_ids = tokenizer.encode(prompt, return_tensors='pt')

output = model.generate(input_ids, max_length=300, num_beams=1)
output_text = tokenizer.decode(output[0])

# textwrap with indentation on every new paragraph
see(output_text)